# Inventory planning

This notebook is aimed to documents the steps taken to build and evaluate a predictive model for inventory planning.<br>
The goal is to predict the quantity of items sold based on historical sales data and various features.<br>
The model will help in making informed decisions for inventory management.

## Data Preparation, Feature Engineering, and Train-Test Split

In this step, we load the historical sales data. This involves:

1. **Loading the Data**: Fetching the data and building clean db in the DuckDB database.
3. **Feature Engineering**: Creating the features to enhance the model's ability to capture patterns in the data.

### Loading and Exploring Data
- **Download Data**: Download data from GDrive.
- **Load Data**: Put the data in a table in DuckDB.

### Feature Engineering

We create new features from existing ones to improve the performance of the model.:

- **Temporal Features**: *is_weekend*, *day_of_week*, *day_of_year*, and *week_of_year* to capture temporal patterns.
- **Lag Features**: *quantity_sold_lag_1* and *quantity_sold_lag_7* to incorporate historical sales data.
- **Rolling Mean**: *rolling_mean_7* to smooth out fluctuations and capture trends.

### Train-Test Split

To ensure that all SKUs are represented in the training set and to maintain the temporal order within each SKU, we perform the following steps:

1. **Data Retrieval**: Fetch the data from the DuckDB database.
2. **SKU-Specific Data**: For each unique SKU, sort the data by date (*dt_submitted*) to maintain temporal order.
3. **Train-Test Split per SKU**: 
   - For SKUs with more than one record, split the data into training and testing sets using an 80-20 split.
   - For SKUs with only one record, include the record in the training set.
4. **Combine Splits**: Aggregate the splits from all SKUs into combined training and testing datasets.

In [1]:
import tasks.prepare

cache_dir = "../.cache/"
con, df_train, df_test = tasks.prepare.get_data(cache_dir=cache_dir)
con.query(query="SELECT * FROM features LIMIT 2").fetchdf()

,sku,dt_submitted,quantity_sold,day,year,month,day_of_week,day_of_year,week_of_year,is_weekend,quantity_sold_lag_1,quantity_sold_lag_7,rolling_mean_7
0,-1001484580476486315,2022-11-26,1,26,2022,11,6,330,47,True,0,0,1.0
1,-1001484580476486315,2023-03-29,1,29,2023,3,3,88,13,False,1,0,1.0


## Model building & Hyperparameter tuning

We build our model by integrating a preprocessing pipeline, hyperparameter tuning and using a time-series aware cross-validation strategy:

### Pipeline and Preprocessing

- **Pipeline**: We use a *Pipeline* to streamline the preprocessing and modeling steps.
- **Preprocessor**: A *ColumnTransformer* is used to handle different preprocessing steps for numerical and categorical features. Numerical features are passed through as is (*passthrough*), while categorical features (*sku*) are one-hot encoded.

### Hyperparameter Tuning with RandomizedSearchCV

- **RandomizedSearchCV**: Instead of an exhaustive grid search, we use *RandomizedSearchCV* to randomly sample a specified number of hyperparameter combinations (n_iter=50) from the defined parameter distribution (*param_dist*). This is more efficient and can still cover a broad search space.

- **Parameter Distribution**: We define a range of values for key hyperparameters of the *XGBRegressor*:
  - *max_depth*: Controls the depth of each tree.
  - *n_estimators*: The number of boosting rounds.
  - *learning_rate*: The step size shrinkage used to prevent overfitting.
  - *subsample*: The fraction of samples to be used for fitting the trees.
  - *colsample_bytree*: The fraction of features to be used for each tree.

### Time Series Cross-Validation
- **TimeSeriesSplit**: We use *TimeSeriesSplit* with 5 splits to ensure that the temporal order of data is respected during cross-validation. This helps in preventing data leakage and provides a more realistic evaluation of the model's performance on future data.


In [2]:
import tasks.train
import helpers.model

y_train = df_train[["quantity_sold"]]
x_train = df_train.drop(labels="quantity_sold", axis=1)

grid_search = helpers.model.get_model()
model = tasks.train.train(
    y_train=y_train,
    x_train=x_train,
    grid_search=grid_search,
)

/Users/badrayour.el-amri/Library/Caches/pypoetry/virtualenvs/inventory-planning-osy1kPWt-py3.11/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


## Model evaluation

After training our model, we evaluate its performance on the test dataset to ensure it generalizes well to unseen data:

- **R-squared (R²)**: Indicates the proportion of variance in the target variable that is predictable from the features.
- **Mean Absolute Error (MAE)**: The average absolute difference between predicted and actual values.
- **Mean Squared Error (MSE)**: The average squared difference between predicted and actual values.
- **Root Mean Squared Error (RMSE)**: The square root of the MSE, providing an error metric in the same units as the target variable.

In [17]:
import tasks.evaluate

y_test = df_test[["quantity_sold"]]
x_test = df_test.drop(labels="quantity_sold", axis=1)
metrics = tasks.evaluate.evaluate(
    model=model,
    x_test=x_test,
    y_test=y_test,
)
print("Quantity sold data distrib:", con.execute("SELECT quantity_sold FROM features").fetch_df().describe().to_dict()["quantity_sold"])
print("Metrics:", metrics)

Quantity sold data distrib: {'count': 1026036.0, 'mean': 6.056740699156755, 'std': 26.17813760095008, 'min': 0.0, '25%': 1.0, '50%': 2.0, '75%': 5.0, 'max': 10040.0}
Metrics: {'r2': 0.7499997019767761, 'mae': 2.6746642945269583, 'mse': 148.86490380974786, 'rmse': 12.201020605250523}


### Understanding the Metrics

1. **R-squared (R²)**:
   - **Value**: `0.75`
   - **Interpretation**: 75% of the variance in the `quantity_sold` can be explained by the model. This is a strong indication that the model has captured patterns in the data.

2. **Mean Absolute Error (MAE)**:
   - **Value**: `2.67`
   - **Interpretation**: On average, our model's predictions are off by about 2.67 units of the `quantity_sold`. Given the mean `quantity_sold` is around 6.06, this is a relatively low error, indicating good predictive accuracy.

3. **Mean Squared Error (MSE)**:
   - **Value**: `148.86`
   - **Interpretation**: There are some larger errors, but this is expected given the variability in the data. The question of outlier processing can be asked.

4. **Root Mean Squared Error (RMSE)**:
   - **Value**: `12.20`
   - **Interpretation**: The typical prediction error is around 12 units. This value is higher than the MAE due to the squaring of errors in the MSE, which penalizes larger errors more heavily.

### Using Metrics to Provide Prediction Intervals

To provide a range for the predicted quantity sold, we can use the Mean Absolute Error (MAE) or Root Mean Squared Error (RMSE).<br>
These metrics help us quantify the uncertainty in our predictions.

#### Prediction Intervals

Prediction intervals give us a range within which the actual quantity sold is likely to fall.<br>
This range is calculated by adding and subtracting the error metric (MAE or RMSE) from the predicted value.

#### MAE
- **Mean Absolute Error (MAE)**: It provides a straightforward way to calculate the prediction interval as it directly reflects the average error.
- **Calculation**: 
  - Lower Bound: `Prediction - MAE`
  - Upper Bound: `Prediction + MAE`

#### RMSE
- **Root Mean Squared Error (RMSE)**: The RMSE gives more weight to larger errors due to the squaring of differences before taking the average. It reflects the typical size of the prediction error in the same units as the target variable.
- **Calculation**: 
  - Lower Bound: `Prediction - RMSE`
  - Upper Bound: `Prediction + RMSE`
